In [6]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset
import torch

- Primero, cargamos un conjunto de datos que contiene preguntas y respuestas sobre ejercicios físicos.


In [7]:
#Cargar el conjunto de datos
data = load_dataset('HazSylvia/Fitness_Unformatted')

#Convertir los datos a un dataset de Hugging Face
dataset = Dataset.from_dict({
    "question": [item["Human"] for item in data["train"]],
    "answer": [item["Assistant"] for item in data["train"]]
})

- Preprocesamos los datos para que estén en un formato adecuado para el entrenamiento del modelo GPT-2. Esto implica la tokenización de las preguntas y respuestas y la configuración de las etiquetas.


In [8]:
#Cargar el modelo y el tokenizador
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 no tiene un token de padding por defecto

#Preprocesar los datos
def preprocess_function(examples):
    inputs = [f"Question: {question} Answer: {answer}" for question, answer in zip(examples["question"], examples["answer"])]
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    labels = model_inputs["input_ids"].clone()
    model_inputs["labels"] = labels
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=["question", "answer"])


Map:   0%|          | 0/928 [00:00<?, ? examples/s]

- Configuramos los argumentos de entrenamiento y utilizamos un `Trainer` para entrenar el modelo GPT-2 con los datos preprocesados.

In [9]:
#Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./gpt2-question-answering",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=1000,
    save_total_limit=3,
    logging_dir="./logs",
)

#Usar DataCollator para el padding dinámico
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # No usamos el enmascarado para modelado de lenguaje
)


In [ ]:
#Entrenar el modelo
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)

trainer.train()

- Una vez finalizado el entrenamiento, guardamos el modelo y el tokenizador para su uso futuro.

In [ ]:
#Guardar el modelo entrenado y el tokenizador
model.save_pretrained("./gpt2-question-answering")
tokenizer.save_pretrained("./gpt2-question-answering")

Función: generate_response(question, model, tokenizer, max_length=512, max_new_tokens=50)

1. Cargar el modelo y el tokenizador entrenados desde la ruta especificada.
    - Ruta del modelo: "./gpt2-question-answering"
    - Cargar el modelo GPT2LMHeadModel desde la ruta especificada.
    - Cargar el tokenizador GPT2Tokenizer desde la misma ruta.
    - Ajustar el token de padding del tokenizador al token EOS.

2. Ajustar el tokenizador para evitar advertencias.
    - Establecer el lado de padding del tokenizador como "left".

3. Generar respuestas 
    - Preparar el prompt incluyendo la pregunta.
    - Tokenizar el prompt y preparar los inputs para el modelo.
    - Generar la respuesta utilizando el modelo entrenado.
    - Decodificar la respuesta y extraer la parte generada.
    - Devolver la respuesta generada.

Uso:
- Hacer una pregunta al modelo y obtener una respuesta generada.
    


In [10]:
#Cargar el modelo y el tokenizador entrenados
model_path = "./gpt2-question-answering"
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token  # Ajustar el token de padding

#Ajustar el tokenizer para evitar advertencias
tokenizer.padding_side = "left"

#Función para generar respuestas
def generate_response(question, model, tokenizer, max_length=512, max_new_tokens=50):
    # Preparar el prompt
    prompt = (
        "You are an expert fitness assistant known for providing clear and concise answers to users' questions. "
        f"Question: {question}"
    )
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding="max_length", max_length=max_length)
    
    # Generar la respuesta
    output = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id  # Asegurar el uso del token de padding correcto
    )
    
    # Decodificar la respuesta
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    # Extraer solo la parte de la respuesta generada
    answer = answer.replace(prompt, "").strip()
    return answer

# Ejemplo de uso: Hacer una pregunta al modelo
question = "What is the best way to gain muscle mass?"
response = generate_response(question, model, tokenizer)

print(f"Question: {question}")
print(response)


Question: What is the best way to gain muscle mass?
Answer: gaining muscle is essential for muscle growth and strength endurance it is important to prioritize lean muscle groups and prioritize fat loss when possible Answer that gaining lean mass can be achieved in a variety of forms including strength training cardio and musclebuilding workouts it can also
